In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movie_data = pd.read_csv("movies.csv")

In [3]:
movie_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movie_data.shape

(4803, 24)

In [5]:
movie_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [6]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director','title']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director', 'title']

In [7]:
# replacing the null valuess with null string

for feature in selected_features:
    movie_data[feature]= movie_data[feature].fillna("")

In [8]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4803 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [9]:
movie_data.fillna("", inplace=True)

In [10]:
combined_features = movie_data["genres"]+ '' + movie_data['keywords']+ '' + movie_data['tagline']+ '' + movie_data['title']+ ''+ movie_data['cast']+ ''+ movie_data['director'] 

In [11]:
combined_features

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiShanghai CallingDaniel...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [12]:
vectorizer = TfidfVectorizer()

In [13]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [14]:
print(feature_vectors)

  (0, 4316)	0.17025221801883436
  (0, 24282)	0.2667685233028317
  (0, 19403)	0.15512064793418184
  (0, 16972)	0.21789049631386714
  (0, 27215)	0.16151542115770343
  (0, 31105)	0.19130283844786003
  (0, 26237)	0.2099225918047785
  (0, 25103)	0.2099225918047785
  (0, 32284)	0.19394534139872813
  (0, 31861)	0.2287225808580658
  (0, 2016)	0.2667685233028317
  (0, 21695)	0.25445469973493695
  (0, 21216)	0.07292947700390828
  (0, 31825)	0.12162833164013258
  (0, 28349)	0.058080304733987974
  (0, 26616)	0.2667685233028317
  (0, 5591)	0.23894111674289278
  (0, 30839)	0.1266090406432436
  (0, 26778)	0.3249592344839965
  (0, 11816)	0.16012420269491529
  (0, 5322)	0.21235345887688567
  (0, 10810)	0.2667685233028317
  (0, 25484)	0.09708587285108575
  (0, 10423)	0.11500505700221522
  (0, 617)	0.08869223925359991
  :	:
  (4801, 32254)	0.2867796942556018
  (4801, 9491)	0.24917675330614356
  (4801, 971)	0.1859984029803072
  (4801, 13728)	0.2867796942556018
  (4801, 21963)	0.2142080096209077
  (4801, 3

In [15]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [16]:
print(similarity)

[[1.         0.07220867 0.01451041 ... 0.         0.         0.        ]
 [0.07220867 1.         0.0253514  ... 0.02825163 0.         0.        ]
 [0.01451041 0.0253514  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.02825163 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


## Getting the movie name from the user


In [17]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : super


In [18]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movie_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [19]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Super', 'Jumper', 'Casper']


In [20]:
close_match = find_close_match[0]
print(close_match)

Super


In [21]:
# finding the index of the movie with title

index_of_the_movie = movie_data[movie_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

4033


In [22]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.006396378391781487), (1, 0.021552479703747705), (2, 0.006761699831045292), (3, 0.029446291377786156), (4, 0.006288251807948789), (5, 0.0063915347676226425), (6, 0.0), (7, 0.0659360805000278), (8, 0.0), (9, 0.036834724350616875), (10, 0.03203825899142464), (11, 0.005815997738494593), (12, 0.0), (13, 0.006041761996796997), (14, 0.060484034841857645), (15, 0.0), (16, 0.06774771969529807), (17, 0.025960186048671858), (18, 0.01123169580656071), (19, 0.005263652503108793), (20, 0.03616404260135666), (21, 0.005709958869296889), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.03930905992646131), (27, 0.005939421381258077), (28, 0.006093398445627457), (29, 0.006759223852983351), (30, 0.03809559868445292), (31, 0.006198767003836967), (32, 0.0), (33, 0.06941810605563964), (34, 0.0), (35, 0.03131091898461049), (36, 0.0057483107016586986), (37, 0.0), (38, 0.0688743955286799), (39, 0.006208786146670177), (40, 0.03078529090618799), (41, 0.006050274869557419), (42, 0.0), (43, 0.0062594319305

In [23]:
len(similarity_score)

4803

In [24]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(4033, 1.0), (174, 0.13744561332205357), (2509, 0.12456504209555973), (4532, 0.12410658889683045), (4436, 0.11132898443345829), (4648, 0.11068387113316244), (1005, 0.10696907293435157), (96, 0.10532176568175963), (101, 0.10456172085392054), (683, 0.10142269240816242), (1553, 0.09725423075123937), (3344, 0.09705876757885551), (1052, 0.09396796987829362), (3394, 0.09306924482470344), (1478, 0.0920105035394678), (94, 0.09076580428146004), (149, 0.0907165378988837), (1416, 0.0898069193831767), (4022, 0.08968977917679191), (4499, 0.08731584914928939), (1038, 0.08414931625925086), (2177, 0.08376653339774826), (1581, 0.08349029992285865), (3334, 0.08240837000167478), (3289, 0.08072420696189726), (3433, 0.08035550992047652), (330, 0.07991602400546709), (1073, 0.07794390045045517), (3643, 0.07791522878463626), (3699, 0.0778849955083554), (329, 0.07695333320459215), (1861, 0.07651374358269847), (198, 0.0759390758556568), (2006, 0.07397547719083958), (1249, 0.07372232725029368), (511, 0.07355221

In [25]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movie_data[movie_data.index==index]['title'].values[0]
    if (i<30):
        print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Super
2 . The Incredible Hulk
3 . Slither
4 . Lonesome Jim
5 . Hard Candy
6 . On the Outs
7 . Traffic
8 . Inception
9 . X-Men: First Class
10 . 3000 Miles to Graceland
11 . Se7en
12 . My Dog Skip
13 . Training Day
14 . Freeheld
15 . A Few Good Men
16 . Guardians of the Galaxy
17 . Armageddon
18 . Jersey Girl
19 . Robot & Frank
20 . Mercy Streets
21 . The Infiltrator
22 . Reign Over Me
23 . Blow
24 . Juno
25 . The Wendell Baker Story
26 . The East
27 . The Lord of the Rings: The Two Towers
28 . The River Wild
29 . Bottle Rocket





### Movie Recommendation Sytem

In [40]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movie_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

if (len(find_close_match) !=0):
    
    close_match = find_close_match[0]

    index_of_the_movie = movie_data[movie_data.title == close_match]['index'].values[0]

    similarity_score = list(enumerate(similarity[index_of_the_movie]))

    sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

    print('Movies suggested for you : \n')
    
    i=1
    
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = movie_data[movie_data.index==index]['title'].values[0]
#         print(title_from_index)
        if (i<16):
            print(i, '.',title_from_index)
            i+=1

else :
    print("Sorry movie not found ")

 Enter your favourite movie name : kgf2
Sorry movie not found 
